In [1]:
# 1. Install Dependencies
# Run this cell once to install the required libraries.
!pip install pandas numpy pathway bokeh

# 2. Import Libraries
import pandas as pd
import numpy as np

# 3. Read the Dataset
# Load the dataset from the specified path.
# We use a comma as the separator.
file_path = 'dataset.csv'
try:
    df = pd.read_csv(file_path)
    print("Dataset loaded successfully!")
except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
    # Create an empty dataframe to prevent further errors
    df = pd.DataFrame()

if not df.empty:
    # --- 4. Preprocessing and Cleaning ---

    # Display initial information about the DataFrame
    print("\nOriginal DataFrame Info:")
    df.info()

    # Display the first 5 rows
    print("\nOriginal Head:")
    print(df.head())

    # a. Clean Column Names
    # To make column names easier to work with, we convert them to snake_case.
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
    print("\nCleaned Column Names:")
    print(df.columns)

    # b. Combine Date and Time
    # Combine the 'lastupdateddate' and 'lastupdatedtime' columns into a single datetime object
    # for easier time-series analysis. We'll name the new column 'timestamp'.
    try:
        df['timestamp'] = pd.to_datetime(df['lastupdateddate'] + ' ' + df['lastupdatedtime'], format='%d-%m-%Y %H:%M:%S')
        # Drop the original date and time columns as they are now redundant.
        df = df.drop(columns=['lastupdateddate', 'lastupdatedtime'])
        print("\nCreated 'timestamp' column.")
    except Exception as e:
        print(f"\nError combining date and time columns: {e}")


    # c. Handle Missing Values
    # Check for any missing values in the dataset.
    print("\nMissing Values Check:")
    print(df.isnull().sum())
    # If there were missing values, you could fill them using methods like:
    # df['queue_length'].fillna(0, inplace=True) # Fill with 0
    # df['traffic_condition_nearby'].fillna(df['traffic_condition_nearby'].mode()[0], inplace=True) # Fill with the mode

    # d. Remove Duplicates
    # Check for and remove any duplicate rows to prevent skewed analysis.
    duplicate_count = df.duplicated().sum()
    print(f"\nNumber of duplicate rows found: {duplicate_count}")
    if duplicate_count > 0:
        df.drop_duplicates(inplace=True)
        print("Duplicate rows have been removed.")


    # e. Drop unnecessary columns
    # The 'id' column is just an index, so we can drop it.
    if 'id' in df.columns:
        df = df.drop(columns=['id'])
        print("\nDropped 'id' column.")


    # Display the cleaned DataFrame
    print("\nCleaned DataFrame Info:")
    df.info()

    print("\nCleaned DataFrame Head:")
    print(df.head())



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 16

Dataset loaded successfully!

Original DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdatedTime         18368 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB

Original Head:
   ID SystemCode

In [2]:
# --- 5. Feature Engineering ---
# This script assumes that the DataFrame 'df' is already loaded and cleaned.

print("\n--- Starting Feature Engineering ---")

# a. Create Occupancy Rate
# This is a core feature for our pricing models.
if 'occupancy' in df.columns and 'capacity' in df.columns:
    # Avoid division by zero if capacity is 0
    df['occupancy_rate'] = df.apply(lambda row: row['occupancy'] / row['capacity'] if row['capacity'] > 0 else 0, axis=1)
    print("Created 'occupancy_rate' feature.")

# b. Extract Time-Based Features
# These help capture daily and weekly patterns.
if 'timestamp' in df.columns:
    df['hour_of_day'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek # Monday=0, Sunday=6
    print("Created 'hour_of_day' and 'day_of_week' features.")

# c. Encode Categorical Variables
# Models require numerical input, so we convert categorical features.

# One-hot encode 'vehicle_type'
if 'vehicle_type' in df.columns:
    try:
        # Using get_dummies to create new columns for each vehicle type
        vehicle_dummies = pd.get_dummies(df['vehicle_type'], prefix='vehicle')
        df = pd.concat([df, vehicle_dummies], axis=1)
        df.drop('vehicle_type', axis=1, inplace=True)
        print("One-hot encoded 'vehicle_type'.")
    except Exception as e:
        print(f"Error encoding 'vehicle_type': {e}")


# The 'is_special_day' is already in a numerical format (0 or 1).

# Note on Normalization/Scaling:
# Features like 'occupancy_rate', 'queue_length', etc., will be normalized
# in the next step, just before building the models. This ensures that
# the scaling is applied correctly based on the training data.

# Note on Proximity to Competitors:
# The calculation for competitor proximity will be handled as part
# of building Model 3, as it requires specific logic for that model.

print("\n--- Feature Engineering Complete ---")

# Display the DataFrame with new features
print("\nDataFrame Info After Feature Engineering:")
df.info()

print("\nDataFrame Head After Feature Engineering:")
print(df.head())




--- Starting Feature Engineering ---
Created 'occupancy_rate' feature.
Created 'hour_of_day' and 'day_of_week' features.

--- Feature Engineering Complete ---

DataFrame Info After Feature Engineering:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   systemcodenumber        18368 non-null  object        
 1   capacity                18368 non-null  int64         
 2   latitude                18368 non-null  float64       
 3   longitude               18368 non-null  float64       
 4   occupancy               18368 non-null  int64         
 5   vehicletype             18368 non-null  object        
 6   trafficconditionnearby  18368 non-null  object        
 7   queuelength             18368 non-null  int64         
 8   isspecialday            18368 non-null  int64         
 9   timestamp              

In [3]:
import numpy as np
import pandas as pd

# --- Model Implementation -- -
# This script contains the functions for the three pricing models.
# It assumes that the DataFrame 'df' has been preprocessed and features have been engineered.


# --- Helper Function for Model 3: Competitor Analysis ---

def calculate_haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on Earth using the Haversine formula.
    This is used in Model 3 to find competitor proximity.
    """
    R = 6371  # Radius of Earth in kilometers

    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

# --- Model 1: Baseline Linear Model ---

def calculate_baseline_linear_price(previous_price, occupancy_rate, alpha=5.0):
    """
    Calculates the next price based on a simple linear function of occupancy.

    Args:
        previous_price (float): The current price of the parking space.
        occupancy_rate (float): The current occupancy rate (Occupancy / Capacity).
        alpha (float): The adjustment factor. This should be tuned for smooth pricing.

    Returns:
        float: The calculated next price.
    """
    # Formula: Price(t+1) = Price(t) + alpha * occupancy_rate
    next_price = previous_price + alpha * occupancy_rate
    return next_price


# --- Model 2: Demand-Based Price Function ---

def calculate_demand_based_price(base_price, data_point, lambda_param=0.5, bounds=(0.5, 2.0)):
    """
    Calculates the price based on a multi-factor demand function.

    NOTE: This function expects the input features in the data_point to be normalized.

    Args:
        base_price (float): The base price for the parking space (e.g., $10).
        data_point (pd.Series): A single row of data for a time step, containing all required features.
        lambda_param (float): The sensitivity parameter for price adjustment.
        bounds (tuple): A tuple containing the lower and upper bounds for the final price
                        as a multiplier of the base price (e.g., (0.5, 2.0) for 0.5x to 2x).

    Returns:
        float: The calculated demand-based price.
    """

    # 1. Define weights for the demand function. These should be tuned.
    weights = {
        'occupancy_rate': 0.40,
        'queue_length': 0.25,
        'traffic_condition_nearby': -0.10, # Negative weight as high traffic might deter parking
        'is_special_day': 0.15,
        'vehicle_car': 0.05,
        'vehicle_truck': 0.10, # Trucks might be charged more
        'vehicle_bike': -0.05, # Bikes might be charged less
        'vehicle_cycle': -0.05
    }

    # 2. Calculate the raw demand score.
    raw_demand = 0
    for feature, weight in weights.items():
        if feature in data_point:
            raw_demand += data_point[feature] * weight

    # 3. Normalize Demand.
    # In a real system, you'd normalize based on the historical min/max of raw_demand.
    # For this simulation, we will use a logistic function to smoothly scale the demand
    # score to a range between -1 and 1, which works well with the pricing formula.
    normalized_demand = (2 / (1 + np.exp(-raw_demand))) - 1

    # 4. Calculate the final price.
    price = base_price * (1 + lambda_param * normalized_demand)

    # 5. Enforce price bounds.
    lower_bound = base_price * bounds[0]
    upper_bound = base_price * bounds[1]
    final_price = np.clip(price, lower_bound, upper_bound)

    return final_price


# --- Model 3: Competitive Pricing Model (Optional) ---

def adjust_price_for_competition(current_price, competitor_prices, adjustment_factor=0.1):
    """
    Adjusts the current price based on the prices of nearby competitors.

    Args:
        current_price (float): The price calculated by your primary model (e.g., Model 2).
        competitor_prices (list[float]): A list of prices for nearby competing lots.
        adjustment_factor (float): How aggressively to react to competitor prices.

    Returns:
        float: The final, competitively-adjusted price.
    """
    if not competitor_prices:
        return current_price

    avg_competitor_price = np.mean(competitor_prices)

    # Logic: If our price is higher than the average, lower it slightly to be more attractive.
    # If our price is lower, we may have room to increase it slightly.
    if current_price > avg_competitor_price:
        # Lower our price, but don't go below the average competitor price
        adjusted_price = max(current_price * (1 - adjustment_factor), avg_competitor_price)
    else:
        # Increase our price, but not beyond the average competitor price
        adjusted_price = min(current_price * (1 + adjustment_factor), avg_competitor_price)

    return adjusted_price




In [4]:
# --- Display Final Results ---
# This script demonstrates how to use the models to generate and adjust prices.
# It assumes the DataFrame 'df' is prepared and the model functions are defined.

print("\n--- Running Pricing Simulation and Displaying Results ---")

# --- 1. Normalization ---
# Model 2 requires normalized features. We will apply Min-Max scaling from scratch.
# Select only the numerical features that need to be scaled for the model.
features_to_normalize = [
    'occupancy_rate', 'queue_length', 'hour_of_day', 'day_of_week',
    'vehicle_car', 'vehicle_truck', 'vehicle_bike', 'vehicle_cycle', 'is_special_day'
]

# Also, handle 'traffic_condition_nearby' by mapping it to numerical values
if 'traffic_condition_nearby' in df.columns:
    traffic_mapping = {'low': 0, 'average': 0.5, 'high': 1.0}
    df['traffic_condition_nearby_encoded'] = df['traffic_condition_nearby'].map(traffic_mapping)
    features_to_normalize.append('traffic_condition_nearby_encoded')

df_normalized = df.copy()
# Apply min-max normalization
for feature in features_to_normalize:
    if feature in df_normalized.columns:
        min_val = df_normalized[feature].min()
        max_val = df_normalized[feature].max()
        if max_val > min_val:
            df_normalized[feature] = (df_normalized[feature] - min_val) / (max_val - min_val)
        else:
            df_normalized[feature] = 0 # Handle cases where all values are the same

print("\nFeatures normalized for Model 2.")

# --- 2. Simulation Setup ---

# Select a specific parking lot to focus on
main_location_code = 'BHMBCCMKT01'
# Select another location to act as a competitor
# We find the next available system code in the dataset
competitor_codes = df['systemcodenumber'].unique()
competitor_location_code = competitor_codes[1] if len(competitor_codes) > 1 else competitor_codes[0]

print(f"\nAnalyzing prices for main location: {main_location_code}")
print(f"Using competitor: {competitor_location_code}")

# Filter the data for our two locations
df_main = df_normalized[df_normalized['systemcodenumber'] == main_location_code].sort_values('timestamp').head(10)
df_competitor = df_normalized[df_normalized['systemcodenumber'] == competitor_location_code]

# Initialize base prices
base_price = 10.0
results = []

# --- 3. Run Simulation ---
for index, main_data_point in df_main.iterrows():
    # Calculate the demand-based price for our main location
    demand_price = calculate_demand_based_price(base_price, main_data_point)

    # Find the competitor's data for the same timestamp
    competitor_data_point = df_competitor[df_competitor['timestamp'] == main_data_point['timestamp']]

    competitor_price = None
    if not competitor_data_point.empty:
        # Calculate the competitor's price for that moment
        competitor_price = calculate_demand_based_price(base_price, competitor_data_point.iloc[0])

    # Adjust the price based on the competitor
    competitive_price = adjust_price_for_competition(demand_price, [competitor_price] if competitor_price else [])

    # Store the results
    results.append({
        'Timestamp': main_data_point['timestamp'],
        'Demand_Based_Price': round(demand_price, 2),
        'Competitor_Price': round(competitor_price, 2) if competitor_price else 'N/A',
        'Final_Competitive_Price': round(competitive_price, 2)
    })

# --- 4. Display Final Results Table ---
results_df = pd.DataFrame(results)

print(f"\n\n--- Competitive Pricing Results for {main_location_code} ---")
print(results_df.to_string())




--- Running Pricing Simulation and Displaying Results ---

Features normalized for Model 2.

Analyzing prices for main location: BHMBCCMKT01
Using competitor: BHMBCCTHL01


--- Competitive Pricing Results for BHMBCCMKT01 ---
            Timestamp  Demand_Based_Price  Competitor_Price  Final_Competitive_Price
0 2016-10-04 07:59:00               10.10             10.30                    10.30
1 2016-10-04 08:25:00               10.10             10.32                    10.32
2 2016-10-04 08:59:00               10.13             10.38                    10.38
3 2016-10-04 09:32:00               10.18             10.50                    10.50
4 2016-10-04 09:59:00               10.25             10.58                    10.58
5 2016-10-04 10:26:00               10.29             10.65                    10.65
6 2016-10-04 10:59:00               10.36             10.76                    10.76
7 2016-10-04 11:25:00               10.41             10.80                    10.80
8 2016-10

In [10]:
import pandas as pd
import numpy as np
from datetime import datetime
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

# --- 0. Helper Functions ---
def calculate_haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def calculate_demand_based_price(base_price, data_point, lambda_param=0.5, bounds=(0.5, 2.0)):
    weights = {
        'occupancy_rate': 0.40, 'queue_length_norm': 0.25, 'traffic_condition_nearby_encoded': -0.10,
        'is_special_day': 0.15, 'vehicle_car': 0.05, 'vehicle_truck': 0.10, 'vehicle_bike': -0.05
    }
    raw_demand = sum(data_point.get(feature, 0) * weight for feature, weight in weights.items())
    normalized_demand = (2 / (1 + np.exp(-raw_demand))) - 1
    price = base_price * (1 + lambda_param * normalized_demand)
    return np.clip(price, base_price * bounds[0], base_price * bounds[1])

def adjust_price_for_competition(current_price, competitor_price, adjustment_factor=0.1):
    if pd.isna(competitor_price):
        return current_price
    if current_price > competitor_price:
        return max(current_price * (1 - adjustment_factor), competitor_price)
    else:
        return min(current_price * (1 + adjustment_factor), competitor_price)

# --- 1. Load Data and Perform ALL Cleaning & Feature Engineering ---

file_path = 'dataset.csv'
try:
    df = pd.read_csv(file_path, sep=',')
except Exception as e:
    raise IOError(f"Failed to read the file '{file_path}'. Please ensure 'dataset.csv' exists and is a valid comma-separated file. Error: {e}")

# Correct the order of operations for cleaning column names.
df.columns = [col.strip() for col in df.columns]
# Apply underscore replacement before converting to lowercase to fix the KeyError.
df.columns = df.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower().str.replace(' ', '_')

# Feature Engineering
df['timestamp'] = pd.to_datetime(df['last_updated_date'] + ' ' + df['last_updated_time'], format='%d-%m-%Y %H:%M:%S')
df.drop(columns=['last_updated_date', 'last_updated_time'], inplace=True)

df['occupancy_rate'] = df['occupancy'] / df['capacity']
df = pd.get_dummies(df, columns=['vehicle_type'], prefix='vehicle')

# Normalization
for feat in ['occupancy', 'queue_length']:
    min_val, max_val = df[feat].min(), df[feat].max()
    df[f'{feat}_norm'] = (df[feat] - min_val) / (max_val - min_val) if max_val > min_val else 0

traffic_mapping = {'low': 0, 'average': 0.5, 'high': 1.0}
df['traffic_condition_nearby_encoded'] = df['traffic_condition_nearby'].map(traffic_mapping)
df.sort_values('timestamp', inplace=True)

# --- 2. Calculate Prices for All Models ---
print("Calculating prices for all models...")

base_price = 10.0
# Model 1
df['model1_price'] = base_price + 5.0 * df['occupancy_rate']

# Model 2
# Ensure all possible vehicle columns exist after get_dummies
for col in ['vehicle_car', 'vehicle_truck', 'vehicle_bike', 'vehicle_cycle']:
    if col not in df.columns:
        df[col] = 0
df['model2_price'] = df.apply(lambda row: calculate_demand_based_price(base_price, row), axis=1)

# Model 3
locations = df[['system_code_number', 'latitude', 'longitude']].drop_duplicates('system_code_number')
competitor_map = {}
for _, row1 in locations.iterrows():
    min_dist = float('inf')
    competitor_id = None
    for _, row2 in locations.iterrows():
        if row1['system_code_number'] != row2['system_code_number']:
            dist = calculate_haversine_distance(row1['latitude'], row1['longitude'], row2['latitude'], row2['longitude'])
            if dist < min_dist:
                min_dist = dist
                competitor_id = row2['system_code_number']
    competitor_map[row1['system_code_number']] = competitor_id

df['competitor_id'] = df['system_code_number'].map(competitor_map)

# Aggregate prices to ensure the lookup index is unique to fix the ValueError.
price_lookup = df.groupby(['timestamp', 'system_code_number'])['model2_price'].mean()

def get_competitor_price(row):
    try:
        return price_lookup.loc[(row['timestamp'], row['competitor_id'])]
    except (KeyError, TypeError):
        return np.nan

df['competitor_price'] = df.apply(get_competitor_price, axis=1)
df['model3_price'] = df.apply(lambda row: adjust_price_for_competition(row['model2_price'], row['competitor_price']), axis=1)
print("Price simulation complete.")

# --- 3. Generate Rerouting Suggestions ---
def generate_rerouting_suggestion(row):
    if row['occupancy_rate'] > 0.95 and not pd.isna(row['competitor_price']) and row['model3_price'] > row['competitor_price']:
        return f"ALERT [{row['timestamp']} | {row['system_code_number']}]: Lot is nearly full. Suggest rerouting to {row['competitor_id']} (Price: ${row['competitor_price']:.2f})"
    return ""
df['suggestion'] = df.apply(generate_rerouting_suggestion, axis=1)

# --- 4. Bokeh Visualization ---
output_notebook()
all_locations = df['system_code_number'].unique()
print(f"\nGenerating static price plots for {len(all_locations)} parking locations...")

for location_code in all_locations:
    location_df = df[df['system_code_number'] == location_code]
    source = ColumnDataSource(location_df)

    p = figure(height=300, width=800, title=f"Simulated Pricing for {location_code}", x_axis_type="datetime", y_axis_label="Price ($)")
    p.line(x='timestamp', y='model1_price', source=source, legend_label="Model 1 (Linear)", color="blue")
    p.line(x='timestamp', y='model2_price', source=source, legend_label="Model 2 (Demand)", color="green")
    p.line(x='timestamp', y='model3_price', source=source, legend_label="Model 3 (Competitive)", color="red", line_width=2)
    p.line(x='timestamp', y='competitor_price', source=source, legend_label="Competitor Price", color="orange", line_dash="dashed")
    p.legend.location = "top_left"
    show(p)

print("\n--- Rerouting Suggestions ---")
suggestions = df[df['suggestion'] != '']
if suggestions.empty:
    print("No rerouting suggestions were generated.")
else:
    for suggestion in suggestions['suggestion']:
        print(suggestion)

Calculating prices for all models...
Price simulation complete.

Generating static price plots for 14 parking locations...



--- Rerouting Suggestions ---
ALERT [2016-11-09 11:27:00 | BHMEURBRD01]: Lot is nearly full. Suggest rerouting to BHMBCCMKT01 (Price: $10.18)
ALERT [2016-12-01 10:58:00 | BHMEURBRD01]: Lot is nearly full. Suggest rerouting to BHMBCCMKT01 (Price: $10.10)
ALERT [2016-12-01 10:58:00 | BHMBCCTHL01]: Lot is nearly full. Suggest rerouting to BHMBCCMKT01 (Price: $10.10)
